<center>
<br />
<h1>Code optimization, Numpy, cython</h1>
<h3>Python</h3>
<br />
<h4>2023</h4> </center>

# Matrix

Create Matrix class:

In [1]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

The simplest implementation of matrix multiplication:

In [2]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

 

Let's check its correctness:

In [3]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))


[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


How much time does it take?

In [4]:
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

In [5]:
%%timeit
matrix_product(*setup())

133 ms ± 4.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Woah! That's too long. <br>
Let's find out where we waste time, for that we can use line_profiler:

In [7]:
#!pip3 install --user line_profiler

# If failed:
# git clone https://github.com/rkern/line_profiler.git
# find line_profiler -name '*.pyx' -exec cython {} \;
# cd line_profiler && pip3 install . --user 

In [6]:
%load_ext line_profiler

In [8]:
%lprun -f matrix_product matrix_product(*setup())

```
Timer unit: 1e-09 s

Total time: 0.308948 s
File: /tmp/ipykernel_932683/1855559380.py
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2         1       2425.0   2425.0      0.0      n_xrows, n_xcols = X.shape
     3         1        380.0    380.0      0.0      n_yrows, n_ycols = Y.shape
     4                                               
     5         1        150.0    150.0      0.0      assert n_xcols == n_yrows, "Incompatible matrix dimensions"
     6                                               
     7         1      21301.0  21301.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     8                                               
     9       100       8863.0     88.6      0.0      for i in range(n_xrows):
    10     10000     784402.0     78.4      0.3          for j in range(n_xcols):
    11   1000000   74426670.0     74.4     24.1              for k in range(n_ycols):
    12   1000000  233703983.0    233.7     75.6                  Z[i][k] += X[i][j] * Y[j][k]
    13         1        111.0    111.0      0.0      return Z
```

We see that a lot of time is spent on for-loop and getting indexes. What do you think can be improved?

In [9]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [10]:
%%timeit
matrix_product(*setup())

119 ms ± 780 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Let's try to remove some element retrieval by index:

In [11]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [13]:
%%timeit
matrix_product2(*setup())

79.9 ms ± 215 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Almost 2 times faster! Since it became about 2 times less operations of retrieval by index.

In [16]:
%lprun -f matrix_product2 matrix_product2(*setup())

```
Timer unit: 1e-09 s

Total time: 0.238204 s
File: /tmp/ipykernel_932683/920470186.py
Function: matrix_product2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product2(X, Y):
     2         1       2785.0   2785.0      0.0      n_xrows, n_xcols = X.shape
     3         1        411.0    411.0      0.0      n_yrows, n_ycols = Y.shape
     4         1      20569.0  20569.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5       100       8633.0     86.3      0.0      for i in range(n_xrows):
     6       100      13303.0    133.0      0.0          Xi = X[i]
     7     10000     735838.0     73.6      0.3          for k in range(n_ycols):
     8     10000     772987.0     77.3      0.3              acc = 0
     9   1000000   75473903.0     75.5     31.7              for j in range(n_xcols):
    10   1000000  159806969.0    159.8     67.1                  acc += Xi[j] * Y[j][k]
    11     10000    1368171.0    136.8      0.6              Z[i][k] = acc
    12         1         90.0     90.0      0.0      return Z
```

Let's go further and try to be even faster!

In [17]:
def matrix_product3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            getX = Xi.__getitem__
            Zi[k] = sum(getX(j) * Ytk[j] for j in range(n_xcols))
    return Z

In [18]:
X = [[1],[2],[3]]
Y = [[4,5,6]]
Z = [[0,0,0],[0,0,0],[0,0,0]]
Yt = [[4],[5],[6]]

for i, (Xi, Zi) in enumerate(zip(X, Z)):
        print((i,Xi,Zi))
        for k, Ytk in enumerate(Yt):
            print((k,Ytk))

(0, [1], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])
(1, [2], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])
(2, [3], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])


In [20]:
%%timeit
matrix_product3(*setup())

63.8 ms ± 354 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Faster!

Conclusion: if your code is slow, do not rush to use other frameworks. Often you can speed up your code at times with little manipulation.

# Numpy

Let's try to take our first implementation, but use an array from numpy instead (we'll see how it works later)

In [8]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [22]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [23]:
%timeit np_matrix_product(*setup_np())

263 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


No profit so far.

In [24]:
X, Y = setup_np()

In [25]:
%timeit np.dot(X, Y)

422 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


It's getting interesting. Good motivation to learn the numpy module.

## Array

In [26]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

print(a.size, a.shape, a.dtype)
print(a)

4 (4,) float64
[4. 5. 2. 3.]


You can see that the array is effectively stored in memory:

In [27]:
print(a.nbytes)
a = np.array([4,1.1,2])
print(a.nbytes)

32
24


Shape can be changed:

In [28]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

a.shape = (2, 2)
a

array([[4., 5.],
       [2., 3.]])

An array is of one type. If you create differently typed elements, they all will be cast to the same type.

In [29]:
np.array([4, 1.1, 2, 3, "string"])

array(['4', '1.1', '2', '3', 'string'], dtype='<U32')

## UFuncs

UFuncs (universal functions) are functions that work element by element.

In [30]:
a = range(4)
b = [value + 1 for value in a]
print(b)

[1, 2, 3, 4]


In numpy:

In [31]:
a = np.arange(4)
b = a + 1
print(b)

[1 2 3 4]


The addition of two arrays.

In [32]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for (i, j) in zip(a, b)]

407 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
a, b = np.array(a), np.array(b)
%timeit a + b

9.92 ms ± 46.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


It's because numpy is written in c.

Other UFuncs

In [34]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

1.36 ms ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

4.78 µs ± 4.42 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


There are many other UFuncs: arithmetic operations, bitwise, comparisons, trigonometric, etc.

## Aggregation

Numpy has many built-in aggregation functions

<img src="aggregations.png" width=600px>

In [36]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

4.86 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
data = [random.random() for _ in range(1000000)]
%timeit np.min(data)

29.3 ms ± 63.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
np_data = np.array(data)
%timeit np.min(np_data)

125 µs ± 4.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Indexing

There are all basic slices and indexes. But there are many additional tasty features:

In [39]:
X = np.arange(6).reshape((2, 3))
X

array([[0, 1, 2],
       [3, 4, 5]])

In [40]:
X[1]

array([3, 4, 5])

In [41]:
X[0, 1]

1

In [42]:
z = X[:1, 1:]
z

array([[1, 2]])

Unlike with standard lists, numpy slices are not copies and you can change the initial list.

In [43]:
z[:1, :1] = 100

In [44]:
X

array([[  0, 100,   2],
       [  3,   4,   5]])

### Masking

In [45]:
a = np.arange(8) ** 2
a[a > 8]

array([ 9, 16, 25, 36, 49])

This works because numpy.array supports boolean mask indexing:

In [46]:
a > 8

array([False, False, False,  True,  True,  True,  True,  True])

You can also use an array of indices:

In [47]:
a = np.arange(4) * 2
print(a)
a[[3, 3, 2, 2, 0, 0, 1, 1]]

[0 2 4 6]


array([6, 6, 4, 4, 0, 0, 2, 2])

## Broadcasting
Or automatic casting of dimensions.

You can stack arrays of different sizes where it makes sense:

In [48]:
a = np.arange(3)

In [49]:
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
b

array([[ 0,  0,  0],
       [10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

In [50]:
b + a

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

But it does not always work as we would like:

In [51]:
X = np.arange(6).reshape((3, 2))
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [52]:
X + np.array([1, 2, 3])

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

Of course, we understand what was meant, but numpy did not understand. We will tell him:

In [54]:
Y = np.array((1, 2, 3))[:, np.newaxis]  # reorient array
print(Y.shape)
print(Y)

(3, 1)
[[1]
 [2]
 [3]]


In [55]:
X + Y

array([[1, 2],
       [4, 5],
       [7, 8]])

Conclusion: Loops in python are slow. It is worth using the built-in numpy functions so the code is more understandable and significant acceleration is gained.

# Cython 
Yet another way to optimize your code

The Cython compiler translates a program written in Python or Cython into more efficient C code

Cython is a Python add-on, supporting C function calls and assignment C language data type variables

Let's see how it works. Let's create a couple of functions in pure python:

In [1]:
def f(x):
    return x**2

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [2]:
%%timeit
integrate_f(0, 10, 1000000)

101 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Enabling Cython magic:

In [2]:
%load_ext Cython

In [4]:
%%cython -a 
def f(x):
    return x**2

def integrate_f2(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython is trying to determine variable types and due to this speed up. But those lines of code where he could not establish the type are highlighted in yellow. Almost everything is yellow.

In [5]:
%%timeit
integrate_f2(0, 10, 1000000)

76.6 ms ± 465 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


It's a bit faster, but we can do better.

Let's apply static typing:

In [6]:
%%cython -a 
def f(double x):
    return x**2

def integrate_f3(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f(a + i * dx)
    return s * dx

In [7]:
%%timeit
integrate_f3(0, 10, 1000000)

30.3 ms ± 426 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Function calls in python can be expensive. For example, in the last example, there was a recurrent conversion from C double to Python float. 

Let's change the prototype of the function f:

In [5]:
%%cython -a 
cpdef double f2(double x):
    return x**2

cpdef double integrate_f4(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f2(a + i * dx)
    return s * dx

In [6]:
%%timeit
integrate_f4(0, 10, 1000000)

627 µs ± 5.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Let's compare with numpy

In [9]:
%%timeit
x = np.linspace(0, 10, 1000000)
(x**2).sum() / 100000

812 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


With Cython you can achieve more than with numpy!

## Matrix
Back to our Matrix example

In [10]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z


The whole thing is yellow!

In [11]:
shape = (100, 100)

In [12]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [13]:
%timeit cy_matrix_product(X, Y)

33.7 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


It is faster than our Python implementation, but so far it does not reach the numpy version. <br>
Let's try cython + numpy

In [15]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [16]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [17]:
%timeit cy_matrix_product(X, Y)

228 ms ± 952 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now it's worse!

Cython can't work with Python imports and now it's required to ask python for every little snippet of code.

Let's explain data types for Cython explicitly.

In [19]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product2(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [21]:
%timeit  cy_matrix_product2(X, Y)

240 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It seems that cython now understands more, but it's still slow, since the main parts are still not clear.

Let's add static typing for input values - now our function takes only values of a specified type.

In [23]:
%%cython -a
import numpy as np
cimport numpy as np  # Not every module may be imported like this


def cy_matrix_product3(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [24]:
%timeit cy_matrix_product3(X, Y)

550 µs ± 648 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Well, now we’ll add quite a bit of black magic. <br>
Add decorators that check the boundaries of arrays and type overflows.

In [26]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product4(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [27]:
%timeit cy_matrix_product4(X, Y)

470 µs ± 254 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Profit.

# Memory Usage (just in case)

In [50]:
#!pip install memory_profiler

In [1]:
%load_ext memory_profiler

## 1) memit

In [2]:
%%memit
a = [0] * 2*10**8
del a

peak memory: 1583.86 MiB, increment: 1526.03 MiB


## 2.1) jupyter mprun

In [3]:
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

%mprun -T mprof0 -f my_func my_func()

ERROR: Could not find file /tmp/ipykernel_935582/2780036313.py


*** Profile printout saved to text file mprof0. 


In [4]:
%%writefile memscript.py
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

Writing memscript.py


In [5]:
from memscript import my_func
%mprun -T mprof0 -f my_func my_func()



*** Profile printout saved to text file mprof0. 


In [6]:
!cat mprof0

Filename: /mnt/storage/lbpe/edu/iimcb-python/IIMCB-Python-2022/Lecture12/memscript.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1     58.7 MiB     58.7 MiB           1   def my_func():
     2     66.2 MiB      7.5 MiB           1       a = [1] * 1000000
     3    135.0 MiB     68.8 MiB           1       b = [2] * 9000000
     4     66.3 MiB    -68.7 MiB           1       del b
     5     66.3 MiB      0.0 MiB           1       return a

## 2.2) Terminal mprun

In [7]:
%%writefile memscript2.py
@profile
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

my_func()

Writing memscript2.py


In [8]:
!cat memscript2.py

@profile
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

my_func()


In [9]:
!python -m memory_profiler memscript2

Filename: /mnt/storage/lbpe/edu/iimcb-python/IIMCB-Python-2022/Lecture12/memscript2.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1   39.945 MiB   39.945 MiB           1   @profile
     2                                         def my_func():
     3   47.578 MiB    7.633 MiB           1       a = [1] * 1000000
     4  116.156 MiB   68.578 MiB           1       b = [2] * 9000000
     5   47.703 MiB  -68.453 MiB           1       del b
     6   47.703 MiB    0.000 MiB           1       return a


